In [4]:
import cv2
import numpy as np
import mediapipe as mp
import time
from collections import deque
import math

# --------- Config ----------
CAM_ID = 0
DRAW_COLOR = (0, 0, 255)   # BGR (red) for strokes
BRUSH_THICKNESS = 6
ERASE_THICKNESS = 50
PINCH_THRESHOLD = 0.05     # normalized distance threshold (tweak per camera)
MAX_POINTS = 10000         # maximum points stored to avoid runaway mem
# --------------------------

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture(CAM_ID)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

# A list of strokes; each stroke is deque of (x,y) points
strokes = []
current_stroke = None

# utility: normalized distance between two landmarks
def norm_dist(lm1, lm2):
    return math.hypot(lm1.x - lm2.x, lm1.y - lm2.y)

# For smoothing small jitters, we can average last N points
SMOOTHING = 3

with mp_hands.Hands(
    model_complexity=1,
    min_detection_confidence=0.6,
    min_tracking_confidence=0.6,
    max_num_hands=1
) as hands:

    prev_time = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Cannot read camera frame. Exiting.")
            break

        frame = cv2.flip(frame, 1)  # mirror image for natural interaction
        h, w, _ = frame.shape

        # Convert to RGB for MediaPipe
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        rgb.flags.writeable = False
        results = hands.process(rgb)
        rgb.flags.writeable = True

        # Transparent canvas for drawing
        canvas = np.zeros_like(frame)

        drawing = False
        pinch = False
        index_pos_px = None

        if results.multi_hand_landmarks:
            hand = results.multi_hand_landmarks[0]
            # landmarks available; index fingertip is id 8, thumb tip id 4
            lm_index = hand.landmark[8]
            lm_thumb = hand.landmark[4]

            # convert normalized to pixel coordinates
            ix, iy = int(lm_index.x * w), int(lm_index.y * h)
            tx, ty = int(lm_thumb.x * w), int(lm_thumb.y * h)
            index_pos_px = (ix, iy)

            # normalized distance (scale-independent)
            d = norm_dist(lm_index, lm_thumb)

            # decide pinch (adjust PINCH_THRESHOLD if needed)
            if d < PINCH_THRESHOLD:
                pinch = True
            else:
                pinch = False

            # draw small circle on detected fingertip
            cv2.circle(frame, (ix, iy), 8, (0, 255, 0) if pinch else (255, 0, 0), -1)

            # draw hand landmarks (optional)
            mp_drawing.draw_landmarks(frame, hand, mp_hands.HAND_CONNECTIONS)

        # Drawing logic: start stroke when pinch==True and index_pos available.
        if index_pos_px and pinch:
            # start or continue stroke
            if current_stroke is None:
                current_stroke = deque(maxlen=MAX_POINTS)
                strokes.append(current_stroke)
            current_stroke.append(index_pos_px)
            drawing = True
        else:
            # release pinch -> finish current stroke
            if current_stroke is not None:
                # remove very short strokes (noise)
                if len(current_stroke) < 3:
                    strokes.pop()
                current_stroke = None

        # draw all strokes on canvas
        for stroke in strokes:
            pts = list(stroke)
            if len(pts) >= 2:
                # optional smoothing average
                for i in range(1, len(pts)):
                    p1 = pts[i-1]
                    p2 = pts[i]
                    cv2.line(canvas, p1, p2, DRAW_COLOR, BRUSH_THICKNESS, cv2.LINE_AA)

        # optionally: draw the current index point (if not drawing)
        if index_pos_px and not drawing:
            cv2.circle(frame, index_pos_px, 5, (0, 255, 255), -1)

        # Combine canvas and frame (alpha blending)
        alpha = 1.0
        beta = 1.0
        gamma = 0
        # overlay: where canvas is non-zero, overlay it
        mask = cv2.cvtColor(canvas, cv2.COLOR_BGR2GRAY)
        _, mask = cv2.threshold(mask, 10, 255, cv2.THRESH_BINARY)
        mask_inv = cv2.bitwise_not(mask)
        frame_bg = cv2.bitwise_and(frame, frame, mask=mask_inv)
        canvas_fg = cv2.bitwise_and(canvas, canvas, mask=mask)
        combined = cv2.add(frame_bg, canvas_fg)

        # HUD — show status and FPS
        cur_time = time.time()
        fps = 1 / (cur_time - prev_time) if prev_time else 0
        prev_time = cur_time

        status_text = f"{'Drawing' if drawing else 'Not drawing'} | FPS: {int(fps)} | Strokes: {len(strokes)}"
        cv2.putText(combined, status_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (230,230,230), 2, cv2.LINE_AA)

        # Controls info
        cv2.putText(combined, "Pinch thumb+index -> Draw", (10, h-20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (200,200,200), 1, cv2.LINE_AA)

        cv2.imshow("Air Writing - press 'c' to clear, 's' to save, 'q' to quit", combined)

        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break
        elif key == ord('c'):  # clear canvas
            strokes = []
            current_stroke = None
        elif key == ord('s'):  # save canvas only
            timestamp = int(time.time())
            save_path = f"air_writing_{timestamp}.png"
            # save the canvas overlay (white background)
            white_bg = np.ones_like(canvas) * 255
            mask = cv2.cvtColor(canvas, cv2.COLOR_BGR2GRAY)
            _, mask = cv2.threshold(mask, 10, 255, cv2.THRESH_BINARY)
            inv = cv2.bitwise_not(mask)
            bg = cv2.bitwise_and(white_bg, white_bg, mask=inv)
            strokes_img = cv2.bitwise_and(canvas, canvas, mask=mask)
            out_img = cv2.add(bg, strokes_img)
            cv2.imwrite(save_path, out_img)
            print(f"Saved: {save_path}")

    cap.release()
    cv2.destroyAllWindows()
